In [17]:
import os
import pathlib
from pathlib import Path
import warnings

import mechanicalsoup
import numpy as np
import pandas as pd
import geopandas as gpd
import pandas_bokeh
import us

# Set root path
root_path = pathlib.Path('.')

# Output notebook for pandas_bokeh
pandas_bokeh.output_notebook()

# Define parquet paths for different data types
data_types = ['alarm_data', 'shapes', 'assignments2010', 'assignments2021', 'blk']
parquet_paths = {data_type: root_path / f'data/{data_type}/final.parquet' for data_type in data_types}

# Create an empty dictionary to store dataframes
dataframes = dict()

# Set state information
state = us.states.CA
print(state.fips, state.name, state.abbr)

# Define constants and CRS (Coordinate Reference Systems)
meters_per_mile = 1609.34
crs = {
    'census': 'EPSG:4269',  # degrees - used by Census
    'area': 'ESRI:102003',  # meters
    'length': 'ESRI:102005',  # meters
}

Loading BokehJS ...

06 California CA


In [2]:
# Define helper functions

def read_existing(data_type):
    """Try to read existing parquet file, return path to raw data if it fails."""
    try:
        dataframes[data_type] = gpd.read_parquet(parquet_paths[data_type])
    except:
        try:
            dataframes[data_type] = pd.read_parquet(parquet_paths[data_type])
        except:
            raw_data_path = parquet_paths[data_type].parent
            raw_data_path.mkdir(exist_ok=True, parents=True)
            return raw_data_path
    return None

import wget

def download_data(file, url):
    """Download and unzip data from internet sources."""
    if not file.is_file():  # check if file already exists
        print(f'Downloading {file} from {url}')
        wget.download(url, str(file))
    
    if file.suffix == ".zip":
        print(f'Unzipping {file}')
        os.system(f'unzip -n {file} -d {file.parent}')


def remove_duplicate_columns(df):
    """Remove duplicate columns (created by joins)."""
    return df.loc[:, ~df.columns.duplicated(keep='first')]


def to_numeric(series):
    """Convert series to numeric if the dtype is not 'geometry'."""
    if series.dtype != 'geometry':
        series = pd.to_numeric(series, errors='ignore', downcast='integer')
    return series


def preprocess_dataframe(df):
    """Preprocess the given dataframe."""
    num_index_columns = len(df.index.names)
    df.reset_index(inplace=True)
    df.columns = [str(col).lower().strip() for col in df.columns]
    df = df.apply(to_numeric)
    index_columns = df.columns[:num_index_columns].tolist()
    return df.set_index(index_columns)

In [3]:
# Read existing or fetch data from sources

data_type = 'alarm_data'
raw_data_path = read_existing(data_type)

if raw_data_path:
    file = raw_data_path / 'alarm_ca_2020_block.csv'
    url = 'https://raw.githubusercontent.com/alarm-redist/census-2020/main/census-vest-2020/ca_2020_block.csv'
    download_data(file, url)

    num_rows = None
    df = preprocess_dataframe(gpd.read_file(file, rows=num_rows))
    column_replacements = {col: col.replace('20', '') if not col.startswith('pre') else col for col in df.columns}

    dataframes[data_type] = preprocess_dataframe(df.rename(columns=column_replacements).set_index('geoid'))
    dataframes[data_type] = dataframes[data_type].iloc[:, :24]

    del df
    dataframes[data_type].to_parquet(parquet_paths[data_type])

In [4]:
dataframes['alarm_data']

,state,county,pop,pop_hisp,pop_white,pop_black,pop_aian,pop_asian,pop_nhpi,pop_other,...,vap_black,vap_aian,vap_asian,vap_nhpi,vap_other,vap_two,pre_16_dem_cli,pre_16_rep_tru,pre__dem_bid,pre__rep_tru
geoid,,,,,,,,,,,,,,,,,,,,,
60014001001000,CA,Alameda County,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60014001001001,CA,Alameda County,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60014001001002,CA,Alameda County,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,26.5,2.2,43.1,4.7
60014001001003,CA,Alameda County,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,285.1,23.4,464,50.3
60014001001004,CA,Alameda County,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61150411021044,CA,Yuba County,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.5,2.9,5.5,4.5
61150411021045,CA,Yuba County,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61150411021046,CA,Yuba County,18,6,2,0,0,1,0,0,...,0,0,0,0,0,4,0.9,1.8,3.3,2.7


In [7]:
data_type = 'shapes'
raw_data_path = read_existing(data_type)

if raw_data_path:
    file = raw_data_path / f'tl_2020_{state.fips}_tabblock20.zip'
    url = f'https://www2.census.gov/geo/tiger/TIGER2020/TABBLOCK20/{file.name}'
    download_data(file, url)

    num_rows = None
    df = preprocess_dataframe(gpd.read_file(file, rows=num_rows))
    column_replacements = {col: col.replace('20', '') for col in df.columns}
    dataframes[data_type] = preprocess_dataframe(df.rename(columns=column_replacements).set_index('geoid')[['aland', 'geometry']])

    del df
    dataframes[data_type].to_parquet(parquet_paths[data_type])

-1 / unknownUnzipping data/shapes/tl_2020_06_tabblock20.zip
Archive:  data/shapes/tl_2020_06_tabblock20.zip
 extracting: data/shapes/tl_2020_06_tabblock20.cpg  
  inflating: data/shapes/tl_2020_06_tabblock20.dbf  
  inflating: data/shapes/tl_2020_06_tabblock20.prj  
  inflating: data/shapes/tl_2020_06_tabblock20.shp  
  inflating: data/shapes/tl_2020_06_tabblock20.shp.ea.iso.xml  
  inflating: data/shapes/tl_2020_06_tabblock20.shp.iso.xml  
  inflating: data/shapes/tl_2020_06_tabblock20.shx  


In [6]:
dataframes['shapes']

,aland,geometry
geoid,,
60650432052005,24205,"POLYGON ((-117.10518 33.60963, -117.10516 33.6..."
60650445072028,7712,"POLYGON ((-116.51852 33.95699, -116.51852 33.9..."
60650427242011,96842,"POLYGON ((-117.14984 33.69644, -117.14892 33.6..."
60650432092003,357212,"POLYGON ((-117.14666 33.56419, -117.14656 33.5..."
60570012112002,64056,"POLYGON ((-120.18434 39.34060, -120.18430 39.3..."
...,...,...
60890123011046,176392,"POLYGON ((-122.33883 40.45689, -122.33881 40.4..."
60890127011111,36156891,"POLYGON ((-121.83545 40.95463, -121.83544 40.9..."
60890125001082,1259952,"POLYGON ((-122.45207 41.08455, -122.45204 41.0..."


In [8]:
data_type = 'assignments2010'
raw_data_path = read_existing(data_type)

if raw_data_path:
    file = raw_data_path / f'BlockAssign_ST{state.fips}_{state.abbr}.zip'
    url = f'https://www2.census.gov/geo/docs/maps-data/data/baf2020/{file.name}'
    download_data(file, url)

    dataframes_list = []
    district_info = {'CD': 'congress2010', 'SLDU': 'senate2010', 'SLDL': 'house2010'}

    for abbr, name in district_info.items():
        district_file = raw_data_path / f'{file.stem}_{abbr}.txt'
        print(f'Reading block → {name} assignments from {district_file}')
        df = preprocess_dataframe(pd.read_csv(district_file, sep='|'))

        if name == 'vtd':
            df['district'] = df['countyfp'].astype(str).str.zfill(3) + df['district'].str.zfill(6)

        column_replacements = {'blockid': 'geoid', 'district': name}
        df = df.rename(columns=column_replacements).set_index('geoid')[[name]]
        dataframes_list.append(df)

    dataframes[data_type] = preprocess_dataframe(pd.concat(dataframes_list, axis=1))
    dataframes[data_type]['fips'] = dataframes[data_type].index // 10000000000

    del dataframes_list
    dataframes[data_type].to_parquet(parquet_paths[data_type])

-1 / unknownUnzipping data/assignments2010/BlockAssign_ST06_CA.zip
Archive:  data/assignments2010/BlockAssign_ST06_CA.zip
  inflating: data/assignments2010/BlockAssign_ST06_CA_SLDL.txt  
  inflating: data/assignments2010/BlockAssign_ST06_CA_AIANNH.txt  
  inflating: data/assignments2010/BlockAssign_ST06_CA_SLDU.txt  
  inflating: data/assignments2010/BlockAssign_ST06_CA_CD.txt  
  inflating: data/assignments2010/BlockAssign_ST06_CA_INCPLACE_CDP.txt  
  inflating: data/assignments2010/BlockAssign_ST06_CA_SDELM.txt  
  inflating: data/assignments2010/BlockAssign_ST06_CA_SDSEC.txt  
  inflating: data/assignments2010/BlockAssign_ST06_CA_SDUNI.txt  
Reading block → congress2010 assignments from data/assignments2010/BlockAssign_ST06_CA_CD.txt
Reading block → senate2010 assignments from data/assignments2010/BlockAssign_ST06_CA_SLDU.txt
Reading block → house2010 assignments from data/assignments2010/BlockAssign_ST06_CA_SLDL.txt


In [9]:
dataframes['assignments2010']

,congress2010,senate2010,house2010,fips
geoid,,,,
60014001001000,13,9,15,6001
60014001001001,13,9,15,6001
60014001001002,13,9,15,6001
60014001001003,13,9,15,6001
60014001001004,13,9,15,6001
...,...,...,...,...
61150411021044,3,4,3,6115
61150411021045,3,4,3,6115
61150411021046,3,4,3,6115


In [10]:
# Final 2021 maps
# https://www.wedrawthelinesca.org/final_maps

data_type = 'assignments2021'
raw_data_path = read_existing(data_type)

if raw_data_path:
    file = raw_data_path / 'CD_Final_equiv.xlsx'
    url = 'https://wedrawthelines.ca.gov/wp-content/uploads/sites/64/2023/01/CD-Final-equiv.xlsx?emrc=63dc56ef11a47'
    download_data(file, url)

    df = pd.read_excel(file, header=None, names=['geoid', 'congress2021'])
    df = preprocess_dataframe(df.set_index('geoid'))
    dataframes[data_type] = df

    del df
    dataframes[data_type].to_parquet(parquet_paths[data_type])

100% [......................................................] 6881671 / 6881671

In [11]:
dataframes['assignments2021']

,congress2021
geoid,
60590635005008,47
60590635005013,47
60590635005009,47
60590635003013,47
60590635003010,47
...,...
60710251002217,25
60710251002235,25
60710251002236,25


In [12]:
def last_step(data_type):
    df = dataframes[data_type]
    df.geometry = df.geometry.to_crs(crs['length'])
    df['perim'] = df.length
    df['area'] = df.area
    df['polsby_popper'] = 4 * np.pi * df['area'] / (df['perim'] ** 2)
    df['density'] = df['pop'] / df['area']

    head_columns = ['fips', 'pop', 'density', 'polsby_popper', 'aland', 'perim']
    tail_columns = ['geometry']
    other_columns = [col for col in df.columns if col not in head_columns + tail_columns]
    all_columns = head_columns + other_columns + tail_columns
    dataframes[data_type] = preprocess_dataframe(df[all_columns].fillna(0))
    dataframes[data_type].to_parquet(parquet_paths[data_type])

In [13]:
# Combine everything at the block level
data_type = 'blk'
raw_data_path = read_existing(data_type)

if raw_data_path:
    dataframes[data_type] = dataframes['alarm_data'].join(dataframes['assignments2010']).join(dataframes['assignments2021']).join(dataframes['shapes'])
    dataframes[data_type] = gpd.GeoDataFrame(dataframes[data_type], geometry='geometry')
    last_step(data_type)

try:
    del dataframes['alarm_data'], dataframes['assignments2010'], dataframes['assignments2021'], dataframes['shapes']
except:
    pass

dataframes[data_type].to_parquet(parquet_paths[data_type])

In [24]:
dataframes['blk']

,fips,pop,density,polsby_popper,aland,perim,state,county,pop_hisp,pop_white,...,pre_16_dem_cli,pre_16_rep_tru,pre__dem_bid,pre__rep_tru,congress2010,senate2010,house2010,congress2021,area,geometry
geoid,,,,,,,,,,,,,,,,,,,,,
60014001001000,6001,0,0,15,51124,2002.271038,CA,Alameda County,0,0,...,0,0,0,0,13,9,15,12,5.085729e+04,"POLYGON ((-2264496.802 203898.399, -2263925.78..."
60014001001001,6001,0,0,10,695414,9163.858276,CA,Alameda County,0,0,...,0,0,0,0,13,9,15,12,6.917906e+05,"POLYGON ((-2265176.643 203948.977, -2264748.29..."
60014001001002,6001,0,0,76,9330,390.010753,CA,Alameda County,0,0,...,26.5,2.2,43.1,4.7,13,9,15,12,9.281641e+03,"POLYGON ((-2264742.631 204058.296, -2264678.98..."
60014001001003,6001,0,0,22,1232350,8185.001155,CA,Alameda County,0,0,...,285.1,23.4,464,50.3,13,9,15,12,1.225929e+06,"POLYGON ((-2265351.277 204250.112, -2265142.87..."
60014001001004,6001,0,0,3,3524,1064.401483,CA,Alameda County,0,0,...,0,0,0,0,13,9,15,12,3.505271e+03,"POLYGON ((-2265292.207 204212.256, -2265209.13..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61150411021044,6115,0,0,10,26265,1798.738057,CA,Yuba County,0,0,...,1.5,2.9,5.5,4.5,3,4,3,3,2.612123e+04,"POLYGON ((-2135423.757 331645.153, -2135263.65..."
61150411021045,6115,0,0,2,0,5035.668539,CA,Yuba County,0,0,...,0,0,0,0,3,4,3,3,5.479830e+04,"POLYGON ((-2134506.065 332033.960, -2134152.18..."
61150411021046,6115,18,0,42,3003346,9424.502353,CA,Yuba County,6,2,...,0.9,1.8,3.3,2.7,3,4,3,3,2.986959e+06,"POLYGON ((-2135400.066 333390.624, -2134797.70..."


In [22]:
def simplify(gdf, tolerance):
    return gdf.set_geometry(gdf.geometry.simplify(tolerance))

dataframes['blk'] = gpd.read_parquet(parquet_paths['blk'])

# simplify polygon boundaries so choropleth renders fast with reasonable file size
dataframes['blk'] = simplify(dataframes['blk'], 100)

# round to make hover tool easier to read
dataframes['blk']['density'] = dataframes['blk']['density'].fillna(0).astype(int)
dataframes['blk']['polsby_popper'] = (dataframes['blk']['polsby_popper'] * 100).astype(int)

In [40]:
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool
from bokeh.palettes import RdYlBu11

# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(dataframes['blk'], geometry='geometry')

# Define the color mapper
color_mapper = LinearColorMapper(palette=RdYlBu11[::-1], low=gdf['congress2021'].min(), high=gdf['congress2021'].max())

# Define the hover tool
hover = HoverTool(tooltips=[('District', '@congress2021'), ('Population', '@pop'), ('Polsby-Popper', '@polsby_popper')])

# Define the GeoJSON data source
geosource = GeoJSONDataSource(geojson=gdf.to_json())

# Define the plot
p = figure(title='California Congressional Districts (2021)', tools=[hover])
p.xaxis.visible = False
p.yaxis.visible = False
p.grid.visible = False

# Add patches to the plot
patches = p.patches('xs', 'ys', source=geosource, fill_color={'field': 'congress2021', 'transform': color_mapper},
                    line_color='black', line_width=0.5, fill_alpha=0.8)

# Add color bar
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8, width=500, height=20, location=(0, 0), orientation='horizontal')
p.add_layout(color_bar, 'below')

# Show the plot
show(p)

In [46]:
#NEEDS FIXING

from bokeh.io import output_file, show
from bokeh.models import GeoJSONDataSource, HoverTool
from bokeh.plotting import figure
from bokeh.palettes import Category10_10

gdf = gpd.GeoDataFrame(dataframes['blk'], geometry='geometry')
ca_src = GeoJSONDataSource(geojson=gdf.to_json())

palette = Category10_10
p = figure(title='California Congressional Districts', width=800, height=600)
p.patches('xs', 'ys', source=ca_src, fill_color={'field': 'congress2021', 'transform': color_mapper})

hover = HoverTool(tooltips=[('Population', '@pop'), ('Polsby-Popper Score', '@polsby_popper'), ('Land Area', '@aland')])
p.add_tools(hover)

show(p)

KeyboardInterrupt: 